# Stackoverflow data

In [ ]:
import random
import numpy as np
import pandas as pd

from sklearn.datasets.lfw import Bunch

## Args

In [12]:
# !kaggle datasets download -d stackoverflow/pythonquestions

In [3]:
df = pd.read_csv('~/.kaggle/datasets/stackoverflow/pythonquestions/Questions.csv', encoding='ISO-8859-1')

In [4]:
df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [5]:
len(df)

607282

In [8]:
df.Title.map(len).max()

172

In [11]:
random.sample(df.Body.tolist(), 10)

['<p>I am running python 3.5 and I want to install pygame, the website only supports up to version 3.2. I have tried a few forums on this and other websites but they are all for python 3.4. Does anyone know were I can i find the install file for python 3.5 and how to install it?</p>\n\n<p>Thanks for your time</p>\n',
 '<pre><code>def show_time(hour,min):\n    hour = int(input())\n    min = \'\'\n    for hour in (hour,min):\n        while 24 &gt;= hour &gt; 12:\n            hour -= 12\n            min == min\n            return hour, ":" min,\'pm\'\n        if hour &lt; 12:\n            if 0 &lt;= min &lt;= 59 \n            hour == hour\n            min == min\n            return hour, ":" min,\'am\'\n</code></pre>\n\n<p>So this is my code so far.^^\nWhen I run this code, i keep getting an infinite loop for one... another thing is.. I feel like i am totally off.   Btw.. I am trying to do this without importing anything and using the string formatting method.  Please HELP!\nHere were my 